In [0]:
from google.colab import files

In [4]:
# IMPORT THE MODEL FILE HERE
model = files.upload()

Saving finalized_model_KNN.sav to finalized_model_KNN.sav


In [6]:
# IMPORT THE DATA HERE
data = files.upload()

Saving hr_data.txt to hr_data.txt


In [49]:
# IMPORT DATA SCALER HERE
scaler = files.upload()

Saving data_scaler_mm.sav to data_scaler_mm.sav


## Code to run

In [0]:
filename_model = 'finalized_model_KNN.sav'
filename_scaler = 'data_scaler_mm.sav'

import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report,mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

loaded_model = pickle.load(open(filename_model, 'rb'))
loaded_scaler = pickle.load(open(filename_scaler, 'rb'))

churn_df = pd.read_csv(list(data)[0])
# Rename column titled 'sales' to 'department' since this makes more sense
churn_df.rename(columns={'sales':'department','average_montly_hours':'avg_monthly_hrs','time_spend_company':'yrs_w_company'},inplace=True)

# The numeric and cts columns
numeric_cols = ['satisfaction_level','last_evaluation','avg_monthly_hrs', 'number_project']
# The categorically numeric/boolean columns
categor_cols = ['yrs_w_company','Work_accident','promotion_last_5years','salary']
# The categorical object columns
dept = ['department']
# Target
target = ['churn']

# Low = 1
# Medium = 2
# High = 3
salary_map = {'low':1,'medium':2,'high':3}
churn_df['salary'] = churn_df['salary'].map(salary_map)

def map_features(dataframe):
  df = dataframe[numeric_cols+categor_cols]
  qc_list = []
  features = df
  for col_i in df.columns:
    for col_j in df.columns:
      new = pd.DataFrame(df[col_i].mul(df[col_j],axis=0))
      new.columns = [col_i+'*'+col_j]
      features = pd.concat([features,new],axis=1)
      if col_i == col_j:
        qc_list.append((col_i,new.sum()))      
  return [qc_list,features]

  def log_column(dataframe):
  '''
  This function applies the log function to all the numeric columns
  '''
  df = dataframe[numeric_cols]
  df.rename(columns = {'satisfaction_level':'log_satisfaction_level',
             'last_evaluation':'log_last_evaluation',
             'avg_monthly_hrs':'log_avg_monthly_hrs',
             'number_project':'log_number_project'}, inplace=True)
  return df.apply(np.log)

  def sqrt_column(dataframe):
  '''
  This function applies the sqrt function to all the numeric columns
  '''
  df = dataframe[numeric_cols]
  df.rename(columns = {'satisfaction_level':'sqrt_satisfaction_level',
             'last_evaluation':'sqrt_last_evaluation',
             'avg_monthly_hrs':'sqrt_avg_monthly_hrs',
             'number_project':'sqrt_number_project'}, inplace=True)
  return df.apply(np.sqrt)

pd.set_option('mode.chained_assignment', None)

analysis_churn_df = pd.concat([log_column(churn_df), sqrt_column(churn_df), map_features(churn_df)[1], churn_df[target]], axis= 1)
analysis_df = pd.concat([log_column(churn_df), sqrt_column(churn_df), map_features(churn_df)[1]], axis= 1)

columns_used = ['log_satisfaction_level', 'yrs_w_company', 'Work_accident', 'salary',
                'satisfaction_level*last_evaluation', 'satisfaction_level*avg_monthly_hrs',
                'satisfaction_level*number_project', 'satisfaction_level*yrs_w_company',
                'satisfaction_level*salary', 'last_evaluation*avg_monthly_hrs',
                'last_evaluation*number_project', 'last_evaluation*yrs_w_company',
                'last_evaluation*salary', 'avg_monthly_hrs*number_project',
                'avg_monthly_hrs*yrs_w_company', 'number_project*yrs_w_company',
                'number_project*salary', 'yrs_w_company*Work_accident']

final_df = analysis_churn_df[columns_used]

x= final_df
x_scaled = loaded_scaler.transform(x)

prediction = pd.DataFrame(loaded_model.predict(x_scaled))
prediction.rename(columns = {0:'Churn Prediction'}, inplace=True)

employee_list = churn_df['id']

employee_prediction = pd.concat([employee_list, prediction], axis =1)

employee_prediction.to_csv('Churn_Prediction.csv')

## To download the prediction file...

Please find the prediction in the files tab to the left. Right click and download the file.